In [1]:
class_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pickle

# Set the necessary parameters
input_shape = (128, 128, 3)  # Input image dimensions
num_classes = 38  # Number of disease classes

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up data augmentation and preprocessing
data_generator = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values

# Load and preprocess the training and testing datasets using the data generator
train_data = data_generator.flow_from_directory('archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train', target_size=(128, 128), batch_size=32, class_mode='categorical')
test_data = data_generator.flow_from_directory('archive/test', target_size=(128, 128), batch_size=32, class_mode='categorical')

# Train the model
model.fit(train_data, epochs=10, validation_data=test_data)

# Save the trained model using pickle
with open('cropheaalth.pkl', 'wb') as file:
    pickle.dump(model, file)



Found 70295 images belonging to 38 classes.
Found 33 images belonging to 1 classes.
Epoch 1/10
2197/2197 [==============================] - 1227s 558ms/step - loss: 1.1408 - accuracy: 0.6670 - val_loss: 543.6508 - val_accuracy: 0.0303
Epoch 2/10
2197/2197 [==============================] - 6145s 3s/step - loss: 0.4084 - accuracy: 0.8712 - val_loss: 665.9278 - val_accuracy: 0.0303
Epoch 3/10
2197/2197 [==============================] - 1218s 555ms/step - loss: 0.2281 - accuracy: 0.9252 - val_loss: 806.4150 - val_accuracy: 0.0303
Epoch 4/10
2197/2197 [==============================] - 1217s 554ms/step - loss: 0.1444 - accuracy: 0.9530 - val_loss: 1026.7629 - val_accuracy: 0.0303
Epoch 5/10
2197/2197 [==============================] - 1222s 556ms/step - loss: 0.0968 - accuracy: 0.9674 - val_loss: 1057.5195 - val_accuracy: 0.0303
Epoch 6/10
2197/2197 [==============================] - 1233s 561ms/step - loss: 0.0783 - accuracy: 0.9737 - val_loss: 1216.6119 - val_accuracy: 0.0303
Epoch 7/10

In [ ]:
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
import pickle

app = Flask(__name__)

# Load the trained model
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((150, 150))
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


# Home page
@app.route('/')
def home():
    return render_template('diseases_identification.html')

# Prediction page
@app.route('/predict', methods=['POST'])
def predict():
    # Get the user image from the request
    user_image = request.files['user_image']
    img = Image.open(user_image)

    # Preprocess the image
    preprocessed_img = preprocess_image(img)

    # Make predictions
    predictions = loaded_model.predict(preprocessed_img)
    predicted_probs = predictions[0]
    predicted_index = np.argmax(predicted_probs)
    predicted_label = class_labels[predicted_index]

    # Render the prediction result
    return render_template('result.html', predicted_label=predicted_label)

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jun/2023 21:49:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2023 21:49:58] "GET /favicon.ico HTTP/1.1" 404 -
[2023-06-24 21:50:08,119] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\admin\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\admin\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\admin\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\admin\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\admin\anaconda3\lib\site-packages\